# Calculation of M/H2/n Queueing System with H2-Warming

### On the Warming Mode
The "warming" mode is activated if the system was empty (no requests were being served and no requests were in queue) and a new request arrived. In this case, the service for the arriving request will start after a random period of "warming" for the system.
Additionally, all incoming requests during the "warming" period of the system are placed in the queue and are not serviced until the end of this period.

### About the Algorithm
In this implementation, it is assumed that the service time and warming time follow *H2*-distributions, each with its own parameters. The parameters of the *H2*-distributions are selected based on the specified average values and coefficients of variation.

The algorithm allows calculating the system for any coefficient of variation in service time. Even when the coefficient of variation in service time is less than 1, the parameters of the approximating *N2*-distribution can be complex, which does not prevent obtaining meaningful results.

For verification, simulation modeling is used.

### Set paramrameters

In [5]:
from most_queue.distr_utils.distribution_fitting import gamma_moments_by_mean_and_cv
from most_queue.general.tables import probs_print, times_print
from most_queue.distributions import GammaDistribution
from most_queue.sim.vacations import VacationQueueingSystemSimulator
from most_queue.theory.vacations.m_h2_h2warm import MH2nH2Warm

n = 5  # number of channels 
l = 1.0  # arrival rate 
utilzation_factor = 0.7  # utilization factor 
b1 = n * utilzation_factor  # Average service time 
b1_warm = n * 0.2  # Average warming time
num_of_jobs = 1000000  # Number of jobs to be served by simulation
b_cv = 1.1  # Coefficient of variation of service time 
b_cv_warm = (
    1.3  # Coefficient of variation of warming time 
)

### Set initial moments of service time

In [2]:
b = gamma_moments_by_mean_and_cv(b1, b_cv)

### Set warm-up moments of service time

In [3]:
b_w = gamma_moments_by_mean_and_cv(b1_warm, b_cv_warm)

### Run simulation

In [4]:
qs = VacationQueueingSystemSimulator(n)  # queueing system simulator object
qs.set_sources(l, 'M')  # set sources with exponential distribution

gamma_params = GammaDistribution.get_params(b)
qs.set_servers(gamma_params, 'Gamma')  # set servers with Gamma distribution

gamma_params_warm = GammaDistribution.get_params(b_w)
qs.set_warm(gamma_params_warm, 'Gamma')  # set warm-up servers with Gamma distribution

sim_results = qs.run(num_of_jobs)  # run simulation

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]10000/1000000:   1%|          | 1/100 [00:00<00:08, 11.08it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:08, 10.98it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:08, 10.98it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:08, 10.98it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.07it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.07it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:08, 11.07it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:08, 10.55it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:08, 10.55it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:08, 10.55it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:08, 10.68it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:08, 10.68it/s]90000/1000000:   9%|▉         | 9/100 [00:00<00:08, 10.68it/s]90000/1000000:  10%|█         | 10/100 [00:00<00:08, 10.71it/s]100000/1000000:  10%|█         | 10/100 [00:00<00:08, 10.71it/s]110000/1000000:  11%|█      

Simulation is finished



### Run numerical method

In [5]:
tt = MH2nH2Warm(n)  # create object of the numerical method

tt.set_sources(l)
tt.set_servers(b, b_w)

calc_results = tt.run()  # run numerical method

num_of_iter = tt.num_of_iter_  # number of iterations

### Print results

In [6]:
print(
    "\nComparison of results calculated by the Takacs-Takaichi method and simulation.\n"
    f"Sim - M/Gamma/{n:^2d} with Gamma warming\n"
    f"Takacs-Takaichi - M/H2/{n:^2d} with H2 warming"
    f"Utilization: {calc_results.utilization:^1.2f}"
)
print(f'Variation coefficient of service time {b_cv:0.3f}')
print(f'Variation coefficient of warming time {b_cv_warm:0.3f}')
print(f"Number of iterations of the Takacs-Takaichi algorithm: {num_of_iter:^4d}")
print(f"Simulation duration: {sim_results.duration:.5f} sec")
print(f"Calculation duration: {calc_results.duration:.5f} sec")

probs_print(sim_results.p, calc_results.p, 10)

times_print(sim_results.v, calc_results.v, is_w=False)


Comparison of results calculated by the Takacs-Takaichi method and simulation.
Sim - M/Gamma/5  with Gamma warming
Takacs-Takaichi - M/H2/5  with H2 warmingUtilization: 0.70
Variation coefficient of service time 1.100
Variation coefficient of warming time 1.300
Number of iterations of the Takacs-Takaichi algorithm:  41 
Simulation duration: 11.95242 sec
Calculation duration: 1.25347 sec
------------------------------------
      Probabilities of states       
------------------------------------
 #  |      Num      |      Sim      
------------------------------------
 0  |   0.021523    |   0.020928    
 1  |   0.086511    |   0.086225    
 2  |    0.15644    |    0.15649    
 3  |    0.18445    |    0.18456    
 4  |    0.16128    |    0.16072    
 5  |    0.11115    |    0.11039    
 6  |   0.078321    |   0.078437    
 7  |   0.055874    |   0.056839    
 8  |   0.040129    |   0.040813    
 9  |   0.028924    |   0.029299    
------------------------------------


Initial moments